Check GPU available

In [7]:
print("GPU available: ", tf.config.list_physical_devices('GPU'))

GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Download essential libraries

In [9]:
!pip install unidecode
!pip install pyvi
!pip install stop-words
!pip install keras_preprocessing
!pip install gensim
!pip install wordcloud
!pip install sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 43.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=965a204f9c08a52cef0070411283edbecfd99f267b61170f56f7e4aa14dbe6ee
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while ge

## Import essential libraries

In [8]:
import re
import unicodedata
import pandas as pd
from unidecode import unidecode
from pyvi import ViTokenizer
import nltk
import string
from gensim.models import Word2Vec
import os
from tqdm import tqdm
import numpy as np
from gensim import models
from stop_words import get_stop_words
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Dropout, Reshape, Flatten, Input, Conv1D, GlobalMaxPooling1D, Embedding
from wordcloud import WordCloud, STOPWORDS
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import MaxPool1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model

## Connect to Google Drive and read CSV file

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd

csv_file_path = '/content/drive/MyDrive/big-data-application/data.csv'

df = pd.read_csv(csv_file_path, encoding='utf-8')

columns_to_keep = ['description', 'ratings']

df = df[columns_to_keep]

df.head()

,description,ratings
0,While perhaps not as celebrated now as some of...,NaN
1,It takes your breath away over 100 years later...,9.0
2,"Wow, is this a memorable film! It is one of th...",9.0
3,"""The Kid"" is a powerfully emotional and wonder...",9.0
4,I've always thought there's a great beauty and...,NaN


## Pre-processing for CNN Model

In [10]:
df = df.dropna(subset=['ratings'])
df.shape

(319197, 2)

In [11]:
import pandas as pd

def determine_label(rating):
    if 1 <= rating <= 4:
        return "NEG"
    elif 5 <= rating <= 7:
        return "NEU"
    elif rating >= 8:
        return "POS"

def determine_value(rating):
    if 1 <= rating <= 4:
        return 0
    elif 5 <= rating <= 7:
        return 1
    elif rating >= 8:
        return 2

print(determine_label(5))
df['label'] = df['ratings'].apply(determine_label)
df['value'] = df['ratings'].apply(determine_value)

df.head(10)

NEU


,description,ratings,label,value
1,It takes your breath away over 100 years later...,9.0,POS,2
2,"Wow, is this a memorable film! It is one of th...",9.0,POS,2
3,"""The Kid"" is a powerfully emotional and wonder...",9.0,POS,2
5,"THE KID (First National Pictures, 1921), a com...",10.0,POS,2
6,Due to an unwed mother abandoning her child in...,9.0,POS,2
7,Charlie Chaplin was perhaps the most innovativ...,8.0,POS,2
8,"It's easy to call Charlie Chaplin a genius, bu...",10.0,POS,2
9,The Kid became a critically hailed internation...,10.0,POS,2
10,What makes a feature film different to a short...,9.0,POS,2
11,It is hard to find such delightful and adorabl...,10.0,POS,2


In [12]:
label_mapping = {"POS": [1, 0, 0], "NEG": [0, 1, 0], "NEU": [0, 0, 1]}

df[['Pos', 'Neg', 'Neu']] = pd.DataFrame(df['label'].map(label_mapping).tolist(), index=df.index)

df.head(5)

,description,ratings,label,value,Pos,Neg,Neu
1,It takes your breath away over 100 years later...,9.0,POS,2,1,0,0
2,"Wow, is this a memorable film! It is one of th...",9.0,POS,2,1,0,0
3,"""The Kid"" is a powerfully emotional and wonder...",9.0,POS,2,1,0,0
5,"THE KID (First National Pictures, 1921), a com...",10.0,POS,2,1,0,0
6,Due to an unwed mother abandoning her child in...,9.0,POS,2,1,0,0


## Word Embedding

In [6]:
all_words = ' '.join(df['description']).lower()

tokenized_words = word_tokenize(all_words)

num_unique_words = set(tokenized_words)

print("Unique words amount: ", num_unique_words)
# num_unique_words = 343252

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(df['description'])
sequences = tokenizer.texts_to_sequences(df['description'])

# Padding sequences
max_length = 100
X = pad_sequences(sequences, maxlen=max_length, padding='post')
X

array([[    8,   282,   127, ...,     0,     0,     0],
       [    7,   146,   192, ...,   573,   239,   857],
       [  809,     3,     1, ...,    76,   737,  1253],
       ...,
       [   23,   243,   163, ...,     4,    49,    12],
       [    3,   958,   454, ...,   330,     2,  1043],
       [  582, 19983,   583, ...,   866,   424,    12]], dtype=int32)

In [15]:
label_names = ['Pos', 'Neg','Neu']
Y = df[label_names].values
Y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

## Devide dataset: train, validation, and test

In [16]:
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## Define CNN Model

In [17]:
cnn_model = Sequential()
cnn_model.add(Embedding(len(tokenizer.word_index) + 1, 300, input_length=100))
cnn_model.add(Conv1D(256, 3, activation='relu', padding='same'))
cnn_model.add(MaxPool1D(2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(128, 3, activation='relu', padding='same'))
cnn_model.add(MaxPool1D(2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(250, activation='relu'))

cnn_model.add(Dense(3, activation='sigmoid'))
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          64483200  
                                                                 
 conv1d (Conv1D)             (None, 100, 256)          230656    
                                                                 
 max_pooling1d (MaxPooling1  (None, 50, 256)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 128)           98432     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 25, 128)           0         
 g1D)                                                   

## Train CNN Model

In [19]:
cnn_model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=5, batch_size=128, verbose=2)

Epoch 1/5
1995/1995 - 127s - loss: 0.2630 - accuracy: 0.8350 - val_loss: 0.2350 - val_accuracy: 0.8580 - 127s/epoch - 64ms/step
Epoch 2/5
1995/1995 - 57s - loss: 0.2137 - accuracy: 0.8698 - val_loss: 0.2378 - val_accuracy: 0.8592 - 57s/epoch - 29ms/step
Epoch 3/5
1995/1995 - 54s - loss: 0.1847 - accuracy: 0.8883 - val_loss: 0.2498 - val_accuracy: 0.8508 - 54s/epoch - 27ms/step
Epoch 4/5
1995/1995 - 52s - loss: 0.1540 - accuracy: 0.9084 - val_loss: 0.2684 - val_accuracy: 0.8458 - 52s/epoch - 26ms/step
Epoch 5/5
1995/1995 - 52s - loss: 0.1224 - accuracy: 0.9285 - val_loss: 0.3163 - val_accuracy: 0.8377 - 52s/epoch - 26ms/step


## Evaluate in test set

In [20]:
result = cnn_model.evaluate(X_test,y_test)
print(dict(zip(cnn_model.metrics_names, result)))

998/998 [==============================] - 4s 4ms/step - loss: 0.3075 - accuracy: 0.8393
{'loss': 0.30751025676727295, 'accuracy': 0.8392543792724609}


## Save Model

In [ ]:
cnn_model.save('sentiment_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Testing an input sentence

In [59]:
text = "Boring movie"

dummy_sequence = tokenizer.texts_to_sequences([text])

dummy_data = pad_sequences(dummy_sequence, maxlen=100, padding='post')

label_predictions = cnn_model.predict(dummy_data, batch_size=128, verbose=2)

print("label_predictions",label_predictions)

text_labels = df['label'].unique()

prediction_label = text_labels[np.argmax(label_predictions[0])]

print(text_labels)
prediction_label

1/1 - 0s - 17ms/epoch - 17ms/step
label_predictions [[0.0041129  0.9610225  0.05703673]]
['POS' 'NEU' 'NEG']


'NEU'